Here we introduce how to customize your own function to use FEM solver to solve the optimization problems you want. The following example is using customize function to solve maximum independent set (MIS) problem using FEM solver. Firstly load all the environments and read the graph instance.

In [1]:
import sys
sys.path.append('../')
from FEM import FEM, read_graph
import torch

num_nodes, num_interactions, couplings = read_graph(
    'instances/mis.txt'
)

Then according to this [paper](https://arxiv.org/pdf/1801.08653.pdf), the MIS problem can be formulated as QUBO type and the target function will be 
$$
E = -\sum_{i}x_i + 2\sum_{(i,j)}x_ix_j
$$
we then encode this target function into the customize expectation function and write the corresponding inference function (which is used to infer the value of target function from the marginal probabilities)

In [2]:
def customize_expected_func(J, p):
    couplings = -torch.eye(J.shape[0], dtype=J.dtype, device=J.device) + 2 * J
    return torch.bmm(
        (p @ couplings).reshape(-1, 1, J.shape[1]),
        p.reshape(-1, p.shape[1], 1)
    ).reshape(-1)

def customize_infer_func(J, p):
    config = p.round()
    return config, customize_expected_func(J, config)

With all prepared, we can use the FEM solver to solve this problem. The method will be choose the `prolem_type` argument to be 'customize' and pass the customized functions into the solver.

In [3]:
num_trials = 100
num_steps = 1000
dev = 'cuda'
case_customize = FEM.from_couplings(
    'customize', num_nodes, num_interactions, couplings,
    customize_expected_func=customize_expected_func,
    customize_infer_func=customize_infer_func
)
case_customize.set_up_solver(num_trials, num_steps, dev=dev)
case_customize.solver.binary = True
config, result = case_customize.solve()
optimal_inds = torch.argwhere(result==result.min()).reshape(-1)
print(f'customize (maximum independent set) test instance, optimal value {result.min()}')
optimal_configs = torch.unique(config[optimal_inds], dim=0)
print('optimal configs are')
for conf in optimal_configs:
    print(conf.cpu().detach().numpy())

customize (maximum independent set) test instance, optimal value -4.0
optimal configs are
[0. 1. 1. 0. 0. 1. 1. 0.]
[1. 0. 0. 1. 1. 0. 0. 1.]


The graph example is from the [wikipedia](https://en.wikipedia.org/wiki/Maximal_independent_set) and looks like ![The graph of the cube has six different maximal independent sets (two of them are maximum), shown as the red vertices.](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Cube-maximal-independence.svg/600px-Cube-maximal-independence.svg.png)

we can see that the optimal configuration found by the FEM solver is the two maximum independent set lie in the middle.